In [18]:
import numpy as np
import math


def PV_cal(F, C, n, m, r):
    return C * (1 - (1 + r/m)**(-n*m)) / (r/m) + F / (1 + r/m)**(n*m)


def sr_cal(P, F, C, n, m, r):
    if C == 0:
        sr = (P/F)**(-1/n)-1
        return sr
    pv = (F+C)/(1+r)**n
    pv = pv/(F+C)
    sr = pv**(-1/n)-1
    sr = sr * m
    return sr


def fr_cal(i, j, sr):
    if i == j:
        return 0
    sr_i = (1+sr[i])**(i+1)
    sr_j = (1+sr[j])**(j+1)
    fr = (sr_j/sr_i)**(1/(j-i))-1
    return fr

In [19]:
# First we calculate YTM

Price = input('Current Bond Price:')
while True:
    try:
        Price = float(Price)
        if Price < 0:
            Price = input('Wrong input. Current Bond Price:')
            continue
        break
    except ValueError:
        Price = input('Wrong input. Current Bond Price:')

Par = input('Bond Par Value:')
while True:
    try:
        Par = float(Par)
        if Par < 0:
            Par = input('Wrong input. Bond Par Value:')
            continue
        break
    except ValueError:
        Par = input('Wrong input. Bond Par Value:')

CouponRate = input('Bond Coupon Rate (% p.a.):')
while True:
    try:
        CouponRate = float(CouponRate)/100
        break
    except ValueError:
        CouponRate = input('Wrong input. Bond Coupon Rate (% p.a.):')

Year = input('Years to Maturity:')
while True:
    try:
        Year = float(Year)
        if Year < 0:
            Year = input('Wrong input. Years to Maturity:')
            continue
        break
    except ValueError:
        Year = input('Wrong input. Years to Maturity:')

PaymentCnt = input('Number of Payments per Year:')
while True:
    try:
        PaymentCnt = int(PaymentCnt)
        if PaymentCnt < 0:
            PaymentCnt = input('Wrong input. Number of Payments per Year:')
            continue
        break
    except ValueError:
        PaymentCnt = input('Wrong input.Number of Payments per Year:')

Coupon = Par * CouponRate / PaymentCnt
ytm = CouponRate
period = int(Year*PaymentCnt)
if ytm == 0.0:
    ytm += 1e-8

while 1:
    pv = PV_cal(Par, Coupon, Year, PaymentCnt, ytm)
    if abs(Price - pv) <= 0.001:
        break
    if Price < pv:
        ytm += 0.0000001
    else:
        ytm -= 0.0000001

ytm = ytm / PaymentCnt
print('\nYield to Maturity (YTM):', str(round(ytm*100, 4))+'%')
ans = [['Input'], ['Presen Value', Price]]
ans.append(['Par Value', Par])
ans.append(['Years to Maturity', Year])
ans.append(['Number of Payments per Year', PaymentCnt])
ans.append([])
ans.append(['YTM', str(round(ytm*100, 4))+'%'])
ans.append([])

Current Bond Price: 1029.1
Bond Par Value: 1000
Bond Coupon Rate (% p.a.): 5
Years to Maturity: 1.5
Number of Payments per Year: 2



Yield to Maturity (YTM): 1.5008%


In [20]:
# Second, we calculate spot rate
spot_rate = sr_cal(Price, Par, Coupon, period, PaymentCnt, ytm)
print('Spot Rate:', str(round(spot_rate*100, 4))+'%')

Spot Rate: 3.0015%


In [21]:
# Third, we calculate foward rate
sr_list = []
for i in range(period-1):
    try:
        sr_list.append(float(input('Spot rate for ' +
                                   str((i+1)/PaymentCnt) +
                                   ' year zero-coupon bond (% p.a.):'))/100)
        continue
    except ValueError:
        sr_list.append(float(input('Wrong input. Spot rate for ' +
                                   str((i+1)/PaymentCnt) +
                                   ' year zero-coupon bond (% p.a.):'))/100)

sr_list.append(spot_rate)

fr = np.zeros((period, period))
for i in range(period):
    for j in range(i, period):
        fr[i, j] = fr_cal(i, j, sr_list)

ans.append(['Spot Rate Table'])
ans.append(['Period'] + [(i+1)/PaymentCnt for i in range(period)])
ans.append(['Spot Rate'] + [str(round(i*100, 4))+'%' for i in sr_list])
ans.append([])
ans.append(['Foward Rate Table'])
ans.append(['From\\To'] + [(i+1)/PaymentCnt for i in range(period)])

for i in range(len(fr)):
    tmp = [(i+1)/PaymentCnt]
    for j in fr[i, :]:
        if j == 0:
            tmp.append('X')
        else:
            tmp.append(str(round(j*100, 4))+'%')
    ans.append(tmp)

Spot rate for 0.5 year zero-coupon bond (% p.a.): 1
Spot rate for 1.0 year zero-coupon bond (% p.a.): 2


In [22]:
import csv
with open('foward_rate.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(ans)